# 🚀 THEBOT - Dashboard Interactif d'Analyse Technique

## Interface Jupyter pour Indicateurs Ultra-Modulaires

**Crypto & Forex Analysis Platform**

- 📊 **Indicateurs disponibles** : SMA, EMA, ATR, RSI
- 🔧 **Architecture ultra-modulaire** validée
- ⚡ **Temps réel** avec signaux automatiques
- 📈 **Graphiques interactifs** Plotly

In [ ]:
# 📦 Import des modules ultra-modulaires THEBOT
import sys
import os
sys.path.append('/workspaces/THEBOT/src')

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from datetime import datetime, timedelta
from decimal import Decimal
import ipywidgets as widgets
from IPython.display import display, HTML

# 🔧 Import des indicateurs ultra-modulaires
from thebot.core.types import MarketData, TimeFrame
from thebot.indicators.basic.sma import SMAIndicator
from thebot.indicators.basic.sma.config import SMAConfig
from thebot.indicators.basic.ema import EMAIndicator
from thebot.indicators.basic.ema.config import EMAConfig
from thebot.indicators.volatility.atr import ATRIndicator
from thebot.indicators.volatility.atr.config import ATRConfig
from thebot.indicators.oscillators.rsi import RSIIndicator
from thebot.indicators.oscillators.rsi.config import RSIConfig

print("🎯 THEBOT Dashboard - Modules chargés avec succès !")
print("📈 Indicateurs disponibles : SMA, EMA, ATR, RSI")
print("🔧 Architecture ultra-modulaire validée")

## 📊 1. Générateur de Données de Marché

Simulation de données crypto/forex réalistes pour démonstration

In [ ]:
def generate_market_data(symbol="BTCUSDT", periods=200, volatility=0.02):
    """
    Génère des données de marché réalistes pour démonstration
    
    Args:
        symbol: Symbole trading (BTCUSDT, EURUSD, etc.)
        periods: Nombre de périodes
        volatility: Niveau de volatilité
    
    Returns:
        Liste de MarketData
    """
    np.random.seed(42)  # Reproductible
    
    # Prix de base selon le symbole
    base_prices = {
        "BTCUSDT": 45000,
        "ETHUSD": 2500,
        "EURUSD": 1.0850,
        "GBPUSD": 1.2650
    }
    
    base_price = base_prices.get(symbol, 100)
    current_price = base_price
    
    data_points = []
    base_time = datetime.now() - timedelta(minutes=periods)
    
    for i in range(periods):
        # Mouvement aléatoire avec tendance légère
        trend = 0.0001 if i > periods // 2 else -0.0001  # Tendance changeante
        change = np.random.normal(trend, volatility)
        current_price *= (1 + change)
        
        # Générer OHLC réaliste
        high = current_price * (1 + abs(np.random.normal(0, volatility/2)))
        low = current_price * (1 - abs(np.random.normal(0, volatility/2)))
        open_price = current_price * (1 + np.random.normal(0, volatility/3))
        
        # S'assurer que OHLC est cohérent
        high = max(high, open_price, current_price)
        low = min(low, open_price, current_price)
        
        # Volume réaliste
        volume = np.random.lognormal(8, 0.5) * 1000  # Volume log-normal
        
        market_data = MarketData(
            timestamp=base_time + timedelta(minutes=i),
            open=Decimal(str(round(open_price, 4))),
            high=Decimal(str(round(high, 4))),
            low=Decimal(str(round(low, 4))),
            close=Decimal(str(round(current_price, 4))),
            volume=Decimal(str(round(volume, 2))),
            timeframe=TimeFrame.M1,
            symbol=symbol
        )
        
        data_points.append(market_data)
    
    return data_points

# 🎲 Générer données d'exemple
market_data = generate_market_data("BTCUSDT", periods=200, volatility=0.025)
print(f"✅ {len(market_data)} points de données générés pour {market_data[0].symbol}")
print(f"📅 De {market_data[0].timestamp} à {market_data[-1].timestamp}")
print(f"💰 Prix : {market_data[0].close} → {market_data[-1].close}")

## 🔧 2. Configuration des Indicateurs Ultra-Modulaires

Widgets interactifs pour paramétrer chaque indicateur

In [ ]:
# 🎛️ Widgets de configuration interactifs

# SMA Configuration
sma_period = widgets.IntSlider(value=20, min=5, max=50, description='SMA Période:')
sma_enable = widgets.Checkbox(value=True, description='Activer SMA')

# EMA Configuration  
ema_period = widgets.IntSlider(value=12, min=5, max=50, description='EMA Période:')
ema_enable = widgets.Checkbox(value=True, description='Activer EMA')

# ATR Configuration
atr_period = widgets.IntSlider(value=14, min=5, max=30, description='ATR Période:')
atr_enable = widgets.Checkbox(value=True, description='Activer ATR')

# RSI Configuration
rsi_period = widgets.IntSlider(value=14, min=5, max=30, description='RSI Période:')
rsi_overbought = widgets.IntSlider(value=70, min=60, max=90, description='RSI Surachat:')
rsi_oversold = widgets.IntSlider(value=30, min=10, max=40, description='RSI Survente:')
rsi_enable = widgets.Checkbox(value=True, description='Activer RSI')

# Symbole et volatilité
symbol_dropdown = widgets.Dropdown(
    options=[('Bitcoin USD', 'BTCUSDT'), ('Ethereum USD', 'ETHUSD'), 
             ('EUR USD', 'EURUSD'), ('GBP USD', 'GBPUSD')],
    value='BTCUSDT',
    description='Symbole:'
)

volatility_slider = widgets.FloatSlider(
    value=0.025, min=0.01, max=0.1, step=0.005,
    description='Volatilité:', readout_format='.1%'
)

# Bouton de mise à jour
update_button = widgets.Button(description='🔄 Mettre à jour Dashboard', 
                              button_style='primary')

# 📋 Interface de configuration
config_box = widgets.VBox([
    widgets.HTML("<h3>🎛️ Configuration des Indicateurs</h3>"),
    widgets.HBox([symbol_dropdown, volatility_slider]),
    widgets.HTML("<b>Moyennes Mobiles</b>"),
    widgets.HBox([sma_enable, sma_period]),
    widgets.HBox([ema_enable, ema_period]),
    widgets.HTML("<b>Volatilité & Momentum</b>"),
    widgets.HBox([atr_enable, atr_period]),
    widgets.HBox([rsi_enable, rsi_period]),
    widgets.HBox([rsi_overbought, rsi_oversold]),
    update_button
])

display(config_box)
print("🎛️ Interface de configuration affichée !")

## 📈 3. Moteur de Calcul Ultra-Modulaire

Traitement temps réel avec tous les indicateurs

In [ ]:
def calculate_all_indicators(market_data, config):
    """
    Calcule tous les indicateurs avec configuration ultra-modulaire
    
    Args:
        market_data: Liste des données de marché
        config: Configuration des indicateurs
    
    Returns:
        DataFrame avec tous les résultats
    """
    results = {
        'timestamp': [],
        'open': [],
        'high': [],
        'low': [],
        'close': [],
        'volume': []
    }
    
    # 🔧 Initialisation des indicateurs ultra-modulaires
    indicators = {}
    
    if config['sma_enable']:
        sma_config = SMAConfig(period=config['sma_period'])
        indicators['sma'] = SMAIndicator(sma_config)
        results['sma'] = []
        results['sma_signals'] = []
    
    if config['ema_enable']:
        ema_config = EMAConfig(period=config['ema_period'])
        indicators['ema'] = EMAIndicator(ema_config)
        results['ema'] = []
        results['ema_signals'] = []
    
    if config['atr_enable']:
        atr_config = ATRConfig(period=config['atr_period'], smoothing_method="ema")
        indicators['atr'] = ATRIndicator(atr_config)
        results['atr'] = []
        results['atr_regime'] = []
    
    if config['rsi_enable']:
        rsi_config = RSIConfig(
            period=config['rsi_period'],
            overbought_level=Decimal(str(config['rsi_overbought'])),
            oversold_level=Decimal(str(config['rsi_oversold']))
        )
        indicators['rsi'] = RSIIndicator(rsi_config)
        results['rsi'] = []
        results['rsi_signals'] = []
        results['rsi_zone'] = []
    
    # 📊 Traitement temps réel de chaque point
    signals_count = {'BUY': 0, 'SELL': 0, 'NEUTRAL': 0}
    
    for data_point in market_data:
        # Données de base
        results['timestamp'].append(data_point.timestamp)
        results['open'].append(float(data_point.open))
        results['high'].append(float(data_point.high))
        results['low'].append(float(data_point.low))
        results['close'].append(float(data_point.close))
        results['volume'].append(float(data_point.volume))
        
        # 🔄 Calcul ultra-modulaire de chaque indicateur
        for name, indicator in indicators.items():
            result = indicator.add_data(data_point)
            
            if name == 'sma':
                value = float(result.value) if result else None
                results['sma'].append(value)
                
                # Signal SMA
                if result:
                    signal = indicator.generate_signal(result)
                    signal_str = signal.direction.value if signal else 'NEUTRAL'
                    results['sma_signals'].append(signal_str)
                    if signal:
                        signals_count[signal.direction.value] += 1
                else:
                    results['sma_signals'].append('NEUTRAL')
            
            elif name == 'ema':
                value = float(result.value) if result else None
                results['ema'].append(value)
                
                # Signal EMA
                if result:
                    signal = indicator.generate_signal(result)
                    signal_str = signal.direction.value if signal else 'NEUTRAL'
                    results['ema_signals'].append(signal_str)
                    if signal:
                        signals_count[signal.direction.value] += 1
                else:
                    results['ema_signals'].append('NEUTRAL')
            
            elif name == 'atr':
                value = float(result.value) if result else None
                results['atr'].append(value)
                
                # Régime de volatilité ATR
                regime = indicator.get_volatility_regime() if result else 'unknown'
                results['atr_regime'].append(regime)
            
            elif name == 'rsi':
                value = float(result.value) if result else None
                results['rsi'].append(value)
                
                # Signal RSI
                if result:
                    signal = indicator.generate_signal(result)
                    signal_str = signal.direction.value if signal else 'NEUTRAL'
                    results['rsi_signals'].append(signal_str)
                    if signal:
                        signals_count[signal.direction.value] += 1
                    
                    # Zone RSI
                    zone = indicator.get_rsi_zone()
                    results['rsi_zone'].append(zone)
                else:
                    results['rsi_signals'].append('NEUTRAL')
                    results['rsi_zone'].append('unknown')
    
    # 📊 Créer DataFrame final
    df = pd.DataFrame(results)
    
    # 📈 Statistiques des signaux
    total_signals = sum(signals_count.values())
    signal_stats = {
        'total': total_signals,
        'buy_ratio': signals_count['BUY'] / max(total_signals, 1),
        'sell_ratio': signals_count['SELL'] / max(total_signals, 1),
        'neutral_ratio': signals_count['NEUTRAL'] / max(total_signals, 1)
    }
    
    return df, signal_stats, indicators

print("⚡ Moteur de calcul ultra-modulaire prêt !")

## 📊 4. Dashboard de Visualisation Interactive

Graphiques Plotly professionnels avec signaux temps réel

In [ ]:
def create_interactive_dashboard(df, signal_stats, indicators, symbol):
    """
    Crée le dashboard interactif complet
    
    Args:
        df: DataFrame avec tous les indicateurs
        signal_stats: Statistiques des signaux
        indicators: Dict des indicateurs actifs
        symbol: Symbole tradé
    
    Returns:
        Figure Plotly interactive
    """
    # 📊 Création des sous-graphiques
    fig = make_subplots(
        rows=4, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=(
            f'🚀 {symbol} - Prix et Moyennes Mobiles',
            '📊 Volume et ATR (Volatilité)',
            '⚡ RSI (Momentum)',
            '🎯 Signaux de Trading'
        ),
        row_heights=[0.4, 0.2, 0.2, 0.2],
        specs=[[{"secondary_y": True}], 
               [{"secondary_y": True}], 
               [{"secondary_y": False}], 
               [{"secondary_y": False}]]
    )
    
    # 🕯️ Chandelier (Prix)
    fig.add_trace(
        go.Candlestick(
            x=df['timestamp'],
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close'],
            name='Prix',
            increasing_line_color='#00ff88',
            decreasing_line_color='#ff4444'
        ),
        row=1, col=1
    )
    
    # 📈 SMA si activé
    if 'sma' in df.columns and 'sma' in indicators:
        valid_sma = df['sma'].dropna()
        if not valid_sma.empty:
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'],
                    y=df['sma'],
                    mode='lines',
                    name=f'SMA({indicators["sma"].config.period})',
                    line=dict(color='blue', width=2)
                ),
                row=1, col=1
            )
    
    # 📈 EMA si activé
    if 'ema' in df.columns and 'ema' in indicators:
        valid_ema = df['ema'].dropna()
        if not valid_ema.empty:
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'],
                    y=df['ema'],
                    mode='lines',
                    name=f'EMA({indicators["ema"].config.period})',
                    line=dict(color='orange', width=2)
                ),
                row=1, col=1
            )
    
    # 📊 Volume
    fig.add_trace(
        go.Bar(
            x=df['timestamp'],
            y=df['volume'],
            name='Volume',
            marker_color='lightblue',
            opacity=0.7
        ),
        row=2, col=1
    )
    
    # 📈 ATR si activé (axe secondaire)
    if 'atr' in df.columns and 'atr' in indicators:
        valid_atr = df['atr'].dropna()
        if not valid_atr.empty:
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'],
                    y=df['atr'],
                    mode='lines',
                    name=f'ATR({indicators["atr"].config.period})',
                    line=dict(color='red', width=2),
                    yaxis='y4'  # Axe secondaire
                ),
                row=2, col=1, secondary_y=True
            )
    
    # ⚡ RSI si activé
    if 'rsi' in df.columns and 'rsi' in indicators:
        valid_rsi = df['rsi'].dropna()
        if not valid_rsi.empty:
            # Ligne RSI
            fig.add_trace(
                go.Scatter(
                    x=df['timestamp'],
                    y=df['rsi'],
                    mode='lines',
                    name=f'RSI({indicators["rsi"].config.period})',
                    line=dict(color='purple', width=2)
                ),
                row=3, col=1
            )
            
            # Niveaux RSI
            overbought = float(indicators['rsi'].config.overbought_level)
            oversold = float(indicators['rsi'].config.oversold_level)
            
            fig.add_hline(y=overbought, line_dash="dash", line_color="red", 
                         annotation_text="Surachat", row=3, col=1)
            fig.add_hline(y=oversold, line_dash="dash", line_color="green", 
                         annotation_text="Survente", row=3, col=1)
            fig.add_hline(y=50, line_dash="dot", line_color="gray", 
                         annotation_text="Neutre", row=3, col=1)
    
    # 🎯 Signaux de trading (heatmap)
    if any(col in df.columns for col in ['sma_signals', 'ema_signals', 'rsi_signals']):
        # Créer matrice des signaux
        signal_cols = [col for col in ['sma_signals', 'ema_signals', 'rsi_signals'] if col in df.columns]
        signal_names = [col.replace('_signals', '').upper() for col in signal_cols]
        
        signal_matrix = []
        for col in signal_cols:
            # Convertir signaux en valeurs numériques
            signal_values = df[col].map({'BUY': 1, 'SELL': -1, 'NEUTRAL': 0}).fillna(0)
            signal_matrix.append(signal_values.tolist())
        
        if signal_matrix:
            fig.add_trace(
                go.Heatmap(
                    z=signal_matrix,
                    x=df['timestamp'],
                    y=signal_names,
                    colorscale=[[0, 'red'], [0.5, 'white'], [1, 'green']],
                    zmid=0,
                    name='Signaux',
                    showscale=True,
                    colorbar=dict(title="Signal", tickvals=[-1, 0, 1], 
                                 ticktext=["SELL", "NEUTRAL", "BUY"])
                ),
                row=4, col=1
            )
    
    # 🎨 Mise en forme du dashboard
    fig.update_layout(
        title=dict(
            text=f"📈 THEBOT Dashboard - {symbol} Analysis",
            font=dict(size=20, color='white'),
            x=0.5
        ),
        height=1000,
        template='plotly_dark',
        showlegend=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    # Axes spécifiques
    fig.update_yaxes(title_text="Prix", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.update_yaxes(title_text="ATR", row=2, col=1, secondary_y=True, side='right')
    fig.update_yaxes(title_text="RSI", row=3, col=1, range=[0, 100])
    fig.update_yaxes(title_text="Indicateurs", row=4, col=1)
    fig.update_xaxes(title_text="Temps", row=4, col=1)
    
    return fig

print("📊 Dashboard de visualisation prêt !")

## 🎯 5. Fonction de Mise à Jour Interactive

In [ ]:
def update_dashboard(b=None):
    """
    Met à jour le dashboard avec les nouvelles configurations
    """
    print("🔄 Mise à jour du dashboard...")
    
    # 📊 Configuration actuelle
    config = {
        'symbol': symbol_dropdown.value,
        'volatility': volatility_slider.value,
        'sma_enable': sma_enable.value,
        'sma_period': sma_period.value,
        'ema_enable': ema_enable.value,
        'ema_period': ema_period.value,
        'atr_enable': atr_enable.value,
        'atr_period': atr_period.value,
        'rsi_enable': rsi_enable.value,
        'rsi_period': rsi_period.value,
        'rsi_overbought': rsi_overbought.value,
        'rsi_oversold': rsi_oversold.value
    }
    
    # 🎲 Régénérer données avec nouvelle configuration
    market_data = generate_market_data(
        symbol=config['symbol'], 
        periods=200, 
        volatility=config['volatility']
    )
    
    # ⚡ Calculer tous les indicateurs
    df, signal_stats, indicators = calculate_all_indicators(market_data, config)
    
    # 📈 Créer dashboard interactif
    fig = create_interactive_dashboard(df, signal_stats, indicators, config['symbol'])
    
    # 📊 Afficher le dashboard
    fig.show()
    
    # 📋 Statistiques des signaux
    print(f"\n📊 STATISTIQUES DES SIGNAUX - {config['symbol']}")
    print(f"Total des signaux: {signal_stats['total']}")
    print(f"🟢 Signaux BUY: {signal_stats['buy_ratio']:.1%}")
    print(f"🔴 Signaux SELL: {signal_stats['sell_ratio']:.1%}")
    print(f"⚪ Signaux NEUTRAL: {signal_stats['neutral_ratio']:.1%}")
    
    # 🔧 État des indicateurs
    print(f"\n🔧 INDICATEURS ACTIFS:")
    for name, indicator in indicators.items():
        metadata = indicator.get_metadata()
        print(f"• {name.upper()}: {metadata['data_points']} points, Prêt: {metadata['is_ready']}")
    
    print("✅ Dashboard mis à jour !")

# 🔗 Connecter le bouton à la fonction
update_button.on_click(update_dashboard)

print("🎯 Fonction de mise à jour connectée !")
print("👆 Cliquez sur 'Mettre à jour Dashboard' pour voir les résultats !")

## 🚀 6. Lancement Initial du Dashboard

Exécution automatique avec configuration par défaut

In [ ]:
# 🎯 Lancement automatique avec configuration par défaut
print("🚀 Lancement initial du Dashboard THEBOT...")
update_dashboard()

print("\n" + "="*60)
print("🎉 DASHBOARD THEBOT OPÉRATIONNEL !")
print("="*60)
print("🔧 Modifiez les paramètres ci-dessus")
print("🔄 Cliquez sur 'Mettre à jour Dashboard' pour voir les changements")
print("📊 Graphiques interactifs Plotly avec zoom/pan")
print("⚡ Signaux temps réel ultra-modulaires")
print("📈 Architecture validée pour 25 indicateurs !")
print("="*60)